# Import libraries

In [3]:
import numpy as np # calculations with arrays
import pandas as pd # user-friendly DataFrames for data representation
import sklearn # machine learning algorithms
from sklearn import ensemble, linear_model
from sklearn.metrics import log_loss

import matplotlib.pyplot as plt # import plot functions
# necessary to plot in jupyter notebook:
%matplotlib inline
import seaborn as sns # make plots beautiful

/home/a/miniconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/home/a/miniconda2/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


# Download data from competition's page

https://inclass.kaggle.com/c/data-mining-in-action-2016-competitions-01/data

# Load data using pandas

In [36]:
train = pd.read_csv('competition_zero/train2.csv')
test = pd.read_csv('competition_zero/test2.csv')
sample_submission = pd.read_csv('competition_zero/sample_submission.csv')

# Data

In [37]:
# print first row
train[:1]

,year,day,team1,team2,score1,score2,target
0,2998,19,317,131,336,278,True


In [38]:
test[:1]

,Id,year,team1,team2
0,0,3021,363,161


In [39]:
# Target variable is "target" and this means we will be predicting it
sample_submission[:1]

,Id,target
0,0,0.5


## Quick look at the unique values in data...

In [40]:
for c in train.columns:
    print c, train[c].unique()[:5]

year [2998 2999 3000 3001 3002]
day [19 28 30 31 33]
team1 [317  61 110 352 229]
team2 [131  29 141 146  91]
score1 [336 301 359 309 332]
score2 [278 259 267 410 220]
target [True False]


# Cross-validation

### Lets split data randomly to train and validatation. We will train our algorithms on selected train set and validate them on validation set. Easy as it can be!

In [41]:
# train size
train.shape 

(101609, 7)

train is quite big, so for example purposes we'll sample only part of it

In [42]:
from sklearn.cross_validation import ShuffleSplit

for itr, ite in ShuffleSplit(len(train), n_iter=1, train_size=0.4, test_size=0.1, random_state=0):
    pass

information about all functions can be found on the internet, for example

http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html

In [ ]:
# or you can open it in you Jupyter notebook executing function in this manner
?ShuffleSplit()

In [43]:
len(itr), len(ite)

(40643, 10161)

In [44]:
itr[:5], ite[:5]

(array([22710, 41665, 91975, 57348, 39931]),
 array([ 37078, 101474,  29858,  61674,   1049]))

now we have validation set "ite" to check the quality of our solution

# features and target

In [45]:
sample_submission[:2]

,Id,target
0,0,0.5
1,1,0.5


we need to change 'target' column in "sample_submission" to our predictions.

For now we will select only features that are present in both train and test:

In [15]:
features = []
for c in train.columns:
    if c in test.columns and c!='target':
        features += [c]
        print '"{}" is present in test and train'.format(c)
    else:
        print '"{}" is NOT present in test'.format(c)
        
features

"year" is present in test and train
"day" is NOT present in test
"team1" is present in test and train
"team2" is present in test and train
"score1" is NOT present in test
"score2" is NOT present in test
"target" is NOT present in test


['year', 'team1', 'team2']

here we split train on "train" and "validation" parts

In [19]:
xtrain = train.loc[itr, features]    
ytrain = train.loc[itr, 'target']

xval = train.loc[ite, features]
yval = train.loc[ite, 'target']

# Baseline solution

lets make baseline first by predicting the mean value

In [17]:
train.target.mean()

0.50096940231672393

In [20]:
constant_prediction = yval * 0 + train.target.mean()
constant_prediction = constant_prediction.values
constant_prediction

array([ 0.5009694,  0.5009694,  0.5009694, ...,  0.5009694,  0.5009694,
        0.5009694])

In [21]:
log_loss(yval, constant_prediction)

0.6931565015839517

In [22]:
submission = sample_submission.copy()
submission.target = train['target'].mean() # notice here that we can refer to a column 'target' in two ways
submission.to_csv('constant_submission.csv', index=False)

Now this should score like "Baseline - Constant" on Leaderboard!
You can submit this by going to 

https://inclass.kaggle.com/c/data-mining-in-action-2016-competitions-01/submissions/attach

# Machine learning

Finally, lets try machine learning!

In [23]:
alg = linear_model.LogisticRegression()
alg.fit(xtrain, ytrain)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [24]:
prediction = alg.predict_proba(xval)[:,1]

In [25]:
log_loss(yval, prediction)

0.69278091662349195

### Well, not so far from the constant solution... Let's try to understand why.

What's a linear model such as LogisticRegression is trying to do is multiply each variable on some coefficient and add add it up, in our case:

y_predicted = column1 \* coef1 + column2 \* coef2 + column3 \* coef3 + bias

We can print coefficients and bias:

In [26]:
alg.coef_, alg.intercept_

(array([[  2.31737378e-07,   3.29139376e-04,  -2.98254396e-04]]),
 array([  5.62890115e-09]))

But clearly, "team1" and "team2" are _categorical_ columns, just like names of the teams. 

So we need to turn "team" columns to something linear algorithm can work with. For example first few rows from here

In [27]:
train.loc[:2, 'team1']

0    317
1     61
2    110
Name: team1, dtype: int64

To this:

In [28]:
pd.get_dummies(train.loc[:2, 'team1'])

,61,110,317
0,0.0,0.0,1.0
1,1.0,0.0,0.0
2,0.0,1.0,0.0


So each team name now has it's own column. Read about "pd.get_dummies" here:

http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html

### But let's come back to more interesting stuff for now
### We are competition's solvers, remember? Lets dive into the space of more complicated models!

In [29]:
alg = ensemble.RandomForestClassifier(15, n_jobs=4)
alg.fit(xtrain, ytrain)
prediction = alg.predict_proba(xval)[:,1]

In [30]:
log_loss(yval, prediction)

1.1744940647416549

Surprisingly, this doesn't work very well. Now, like competition pro, let's make our models bigger!

In [31]:
alg = ensemble.RandomForestClassifier(150, n_jobs=4)
alg.fit(xtrain, ytrain)
prediction = alg.predict_proba(xval)[:,1]

In [32]:
log_loss(yval, prediction)

0.74131637899967551

### Almost there! But for now let's skip this model too and go to _real_ competitions stuff

In [33]:
import xgboost

In [34]:
param = {}
param['max_depth'] = 8
param['booster'] = 'gbtree'
param['objective'] = 'binary:logistic'
param['eval_metric'] = 'logloss'
param['eta'] = 0.1

numround = 100

Xgboost parameters

https://github.com/dmlc/xgboost/blob/master/doc/parameter.md

In [35]:
Xdatatrain = xgboost.DMatrix(data = xtrain, label = ytrain)
Xdatatest = xgboost.DMatrix(data = xval, label = yval)

plst = list(param.items())
watchlist = [(Xdatatrain, 'train'), (Xdatatest, 'eval')]            

bst = xgboost.train(plst, Xdatatrain, numround, evals = watchlist, verbose_eval = 10)
# ypredxgb_tr = bst.predict(Xdatatrain)

[0]	train-logloss:0.689826	eval-logloss:0.691345
[10]	train-logloss:0.670759	eval-logloss:0.681402
[20]	train-logloss:0.658005	eval-logloss:0.67538
[30]	train-logloss:0.647507	eval-logloss:0.669858
[40]	train-logloss:0.638459	eval-logloss:0.66616
[50]	train-logloss:0.628218	eval-logloss:0.661736
[60]	train-logloss:0.619701	eval-logloss:0.658285
[70]	train-logloss:0.6126	eval-logloss:0.655751
[80]	train-logloss:0.604423	eval-logloss:0.652431
[90]	train-logloss:0.597331	eval-logloss:0.649311


Wow! Finally our model better than constant predictions! Congratulations! Don't hesitate, submit!

In [95]:
ss = sample_submission.copy()

ss.target = bst.predict(xgboost.DMatrix(test[features]))
ss.to_csv('mighty_xgboost.csv', index=False)

### Strange, but it seems like we got 0.658 instead of 0.649! 

### What could it be? Perhabs we need to train on all data instead of just 40% of it? Or may be should think over our cross-validation process?

### Let's overview now what we just did here:
1) made cross-validation

2) tried linear models, they didn't work, but we figured out how to tackle this problem

3) tried random forest and almost beat constant benchmark

4) tried xgboost and finally beat constant prediction!

### But there is the last thing you must know before you'll start this challenge by trying to make the most thorough parameter tuning: the data has it's secrets and those who will find them will be generously rewarded...

### now, good luck with it!